# torchtext 튜토리얼

## 샘플 데이터셋 다운로드

In [155]:
import pandas as pd

df = pd.read_csv('movie.csv')
df

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1


## 토크나이저 생성

In [156]:
from torchtext.data.utils import get_tokenizer

tokenizer의 타입으로는 `basic_english`, `spacy`, `moses`, `toktok`, `revtok`, `subword` 이 있습니다.

다만, 이 중 몇개의 타입은 추가 패키지가 설치되어야 정상 동작합니다.

In [157]:
tokenizer = get_tokenizer('basic_english', language='en')
tokenizer("I'd like to learn torchtext")

['i', "'", 'd', 'like', 'to', 'learn', 'torchtext']

토큰 타입을 지정하면 그에 맞는 tokenizer를 반환하는 함수를 생성한 뒤 원하는 타입을 지정하여 tokenizer를 생성할 수 있습니다.

In [158]:
def generate_tokenizer(tokenizer_type, language='en'):
    return get_tokenizer(tokenizer_type, language=language)

`basic_english`를 적용한 경우

In [159]:
tokenizer = generate_tokenizer('basic_english')
tokenizer("I'd like to learn torchtext")

['i', "'", 'd', 'like', 'to', 'learn', 'torchtext']

`toktok`을 적용한 경우

In [160]:
tokenizer = generate_tokenizer('toktok')
tokenizer("I'd like to learn torchtext")

['I', "'", 'd', 'like', 'to', 'learn', 'torchtext']

In [161]:
from nltk.tokenize import word_tokenize

word_tokenize("I'd like to learn torchtext")

['I', "'d", 'like', 'to', 'learn', 'torchtext']

## 필드(Field) 정의

In [10]:
from torchtext.legacy import data

`torchtext.legacy.data.Field` 
- `Field` 클래스는 `Tensor`로 변환하기 위한 지침과 함께 데이터 유형을 정의합니다. 
- `Field` 객체는 `vocab` 개체를 보유합니다.
- `Field` 객체는 토큰화 방법, 생성할 Tensor 종류와 같이 데이터 유형을 수치화하는 역할을 수행합니다.

In [162]:
TEXT = data.Field(sequential=True,    # 순서를 반영
                  tokenize=tokenizer, # tokenizer 지정
                  fix_length=120,     # 한 문장의 최대 길이 지정
                  lower=True,         # 소문자 화
                  batch_first=True)   # batch 를 가장 먼저 출력


LABEL = data.Field(sequential=False)

`fields` 변수에 dictionary를 생성합니다.
- `key`: 읽어 들여올 파일의 열 이름을 지정합니다.
- `value`: (`문자열`, `data.Field`) 형식으로 지정합니다. 여기서 지정한 문자열이 나중에 생성된 data의 변수 이름으로 생성됩니다.

In [163]:
fields = [('text', TEXT), 
          ('label', LABEL)]

## 데이터셋 로드 및 분할

`TabularDataset` 클래스는 정형 데이터파일로부터 직접 데이터를 읽을 때 유용합니다.

지원하는 파일 형식은 `CSV`, `JSON`, `TSV` 을 지원합니다.

In [164]:
import random
from torchtext.legacy.data import TabularDataset

SEED = 123

dataset = TabularDataset(path='movie.csv',  # 파일의 경로
                         format='CSV',         # 형식 지정
                         fields=fields,  # 이전에 생성한 field 지정
                         skip_header=True
                        )        

이전에 생성한 `dataset` 변수로 train / test 데이터셋을 분할 합니다.

In [165]:
train_data, test_data = dataset.split(split_ratio=0.8,               # 분할 비율
                                      stratified=True,               # stratify 여부
                                      strata_field='label',          # stratify 대상 컬럼명
                                      random_state=random.seed(SEED) # 시드
                                     )

In [166]:
# 생성된 train / test 데이터셋의 크기를 출력 합니다.
len(train_data), len(test_data)

(32000, 8000)

## 단어 사전 생성

In [167]:
TEXT.build_vocab(train_data, 
                 max_size=2000,  # 최대 vocab_size 지정
                 min_freq=5,     # 최소 빈도 단어수 지정
                 vectors='glove.6B.100d')   # 워드임베딩 vector 지정, None으로 지정시 vector 사용 안함

LABEL.build_vocab(train_data)

In [168]:
NUM_VOCABS = len(TEXT.vocab.stoi)
NUM_VOCABS

2002

`TEXT.vocab.stoi`는 문자열을 index로, `TEXT.vocab.itos`는 index를 문자열로 변환합니다.

In [169]:
TEXT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x7f588d30b370>>,
            {'<unk>': 0,
             '<pad>': 1,
             'the': 2,
             ',': 3,
             'and': 4,
             'a': 5,
             'of': 6,
             'to': 7,
             "'": 8,
             'is': 9,
             '/': 10,
             'in': 11,
             'it': 12,
             'i': 13,
             'this': 14,
             'that': 15,
             '"': 16,
             's': 17,
             '><br': 18,
             'was': 19,
             'as': 20,
             'for': 21,
             'with': 22,
             'but': 23,
             'movie': 24,
             ')': 25,
             'film': 26,
             'you': 27,
             't': 28,
             'on': 29,
             '(': 30,
             'not': 31,
             'are': 32,
             'he': 33,
             'his': 34,
             '.': 35,
             'have': 36,
             'be': 37,
     

In [171]:
# string to index
print(TEXT.vocab.stoi['this'])
print(TEXT.vocab.stoi['pretty'])
print(TEXT.vocab.stoi['original'])

print('==='*10)

# index to string
print(TEXT.vocab.itos[14])
print(TEXT.vocab.itos[194])
print(TEXT.vocab.itos[237])

14
194
237
this
pretty
original


## 버킷 이터레이터 생성

- `BucketIterator` 의 주된 역할은 데이터셋에 대한 배치 구성입니다.

In [172]:
import torch

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 32

train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data),     # dataset
    sort=False,
    repeat=False,
    batch_size=BATCH_SIZE,       # 배치사이즈
    device=device)               # device 지정

1개의 배치를 추출합니다.

In [232]:
# 1개의 batch 추출
sample_data = next(iter(train_iterator))

`text` 의 shape 를 확인합니다.

In [233]:
# batch_size, sequence_length
sample_data.text.shape

torch.Size([32, 120])

`label` 의 shape 를 확인합니다.

In [234]:
# batch_size
sample_data.label.shape

torch.Size([32])

In [235]:
sample_data.label

tensor([1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1,
        2, 1, 2, 1, 1, 1, 1, 2], device='cuda:1')

In [236]:
sample_data.text.max()

tensor(1997, device='cuda:1')

## 모델 생성

In [237]:
from tqdm import tqdm  # Progress Bar 출력
import numpy as np
import torch.nn as nn
import torch.optim as optim


class TextClassificationModel(nn.Module):
    def __init__(self, num_classes, vocab_size, embedding_dim, hidden_size, num_layers, seq_length, drop_prob=0.1):
        super(TextClassificationModel, self).__init__()
        self.num_classes = num_classes 
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.seq_length = seq_length
        
        self.embedding = nn.Embedding(num_embeddings=vocab_size, 
                                      embedding_dim=embedding_dim)
        
        self.lstm = nn.LSTM(input_size=embedding_dim, 
                            hidden_size=hidden_size, 
                            num_layers=num_layers)
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.output = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x, hidden_and_cell):
        x = self.embedding(x)
        output, (h, c) = self.lstm(x, hidden_and_cell)
        h = output[:, -1, :]
        h = h.reshape(-1, self.hidden_size)
        h = self.dropout(h)
#         out = self.relu(self.fc(h))
        return self.output(h)

In [243]:
config = {
    'num_classes': 2, 
    'vocab_size': NUM_VOCABS,
    'embedding_dim': 128, 
    'hidden_size': 256, 
    'num_layers': 1, 
    'seq_length': 120
}

model = TextClassificationModel(**config)
model.to(device)

TextClassificationModel(
  (embedding): Embedding(2002, 128)
  (lstm): LSTM(128, 256)
  (dropout): Dropout(p=0.1, inplace=False)
  (output): Linear(in_features=256, out_features=2, bias=True)
)

In [244]:
# loss 정의: CrossEntropyLoss
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저 정의: bert.paramters()와 learning_rate 설정
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [245]:
def model_train(model, data_loader, loss_fn, optimizer, config, device):
    # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
    model.train()
    
    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0
    counts = 0
    
    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)
    
    # mini-batch 학습을 시작합니다.
    for idx, data in enumerate(prograss_bar):
        # text, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        text = data.text.to(device)
        label = data.label.to(device)
        label.data.sub_(1)
        
        # 누적 Gradient를 초기화 합니다.
        optimizer.zero_grad()
        
        initial_hidden = torch.zeros(config['num_layers'], config['seq_length'], config['hidden_size']).to(device)
        initial_cell = torch.zeros(config['num_layers'], config['seq_length'], config['hidden_size']).to(device)
        
        # Forward Propagation을 진행하여 결과를 얻습니다.
        output = model(text, (initial_hidden, initial_cell))
        
        # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
        loss = loss_fn(output, label)
        
        # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
        loss.backward()
        
        # 계산된 Gradient를 업데이트 합니다.
        optimizer.step()
        
        # output의 max(dim=1)은 max probability와 max index를 반환합니다.
        # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
        _, pred = output.max(dim=1)
        
        # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
        # 합계는 corr 변수에 누적합니다.
        corr += pred.eq(label).sum().item()
        counts += len(label)
        
        # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
        # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
        # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
        running_loss += loss.item() * label.size(0)
        
        # 프로그레스바에 학습 상황 업데이트
        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")
        
    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)
    
    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader), acc

In [246]:
def model_evaluate(model, data_loader, loss_fn, config, device):
    # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다. 
    # dropout과 같은 layer의 역할 변경을 위하여 evaluation 진행시 꼭 필요한 절차 입니다.
    model.eval()
    
    # Gradient가 업데이트 되는 것을 방지 하기 위하여 반드시 필요합니다.
    with torch.no_grad():
        # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
        corr = 0
        running_loss = 0
        
        # 배치별 evaluation을 진행합니다.
        for data in data_loader:
            # text, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
            text = data.text.to(device)
            label = data.label.to(device)
            label.data.sub_(1)
            
            initial_hidden = torch.zeros(config['num_layers'], config['seq_length'], config['hidden_size']).to(device)
            initial_cell = torch.zeros(config['num_layers'], config['seq_length'], config['hidden_size']).to(device)
            
            # 모델에 Forward Propagation을 하여 결과를 도출합니다.
            output = model(text, (initial_hidden, initial_cell))
            
            # output의 max(dim=1)은 max probability와 max index를 반환합니다.
            # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
            _, pred = output.max(dim=1)
            
            # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
            # 합계는 corr 변수에 누적합니다.
            corr += torch.sum(pred.eq(label)).item()
            
            # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
            # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
            # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
            running_loss += loss_fn(output, label).item() * label.size(0)
        
        # validation 정확도를 계산합니다.
        # 누적한 정답숫자를 전체 데이터셋의 숫자로 나누어 최종 accuracy를 산출합니다.
        acc = corr / len(data_loader)
        
        # 결과를 반환합니다.
        # val_loss, val_acc
        return running_loss / len(data_loader), acc

In [247]:
# 최대 Epoch을 지정합니다.
num_epochs = 10

# checkpoint로 저장할 모델의 이름을 정의 합니다.
model_name = 'LSTM-Text-Classification'

min_loss = np.inf

# Epoch 별 훈련 및 검증을 수행합니다.
for epoch in range(num_epochs):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    train_loss, train_acc = model_train(model, train_iterator, loss_fn, optimizer, config, device)

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = model_evaluate(model, test_iterator, loss_fn, config, device)   
    
    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(model.state_dict(), f'{model_name}.pth')
    
    # Epoch 별 결과를 출력합니다.
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

training loss: 22.20417, training accuracy: 0.51069: 100% 1000/1000 [00:04<00:00, 223.77batch/s]


[INFO] val_loss has been improved from inf to 22.15879. Saving Model!
epoch 01, loss: 22.20417, acc: 0.51069, val_loss: 22.15879, val_accuracy: 16.96800


training loss: 22.08324, training accuracy: 0.53194: 100% 1000/1000 [00:04<00:00, 221.56batch/s]


epoch 02, loss: 22.08324, acc: 0.53194, val_loss: 22.25570, val_accuracy: 16.53200


training loss: 21.92497, training accuracy: 0.54166: 100% 1000/1000 [00:04<00:00, 222.25batch/s]


...

epoch 08, loss: 21.29710, acc: 0.56131, val_loss: 22.47132, val_accuracy: 16.75200


training loss: 21.23197, training accuracy: 0.56222: 100% 1000/1000 [00:04<00:00, 223.01batch/s]


epoch 09, loss: 21.23197, acc: 0.56222, val_loss: 22.82309, val_accuracy: 16.66400


training loss: 21.19722, training accuracy: 0.56316: 100% 1000/1000 [00:04<00:00, 223.63batch/s]


epoch 10, loss: 21.19722, acc: 0.56316, val_loss: 22.81967, val_accuracy: 16.66800
